In [2]:
!ls

Generate decoys.ipynb
Select and filter Class II MHC mass spec data from IEDB.ipynb
iedb-mhc2-ligands-7-29-2018.csv


In [3]:
hits = pd.read_csv("iedb-mhc2-ligands-7-29-2018.csv")

In [11]:
hit_seqs = set(hits.peptide)


In [17]:
lengths = [len(p) for p in hit_seqs]
lengths_set = set(lengths)
max_length = max(lengths_set)
length_counts = np.ones(max_length, dtype=int)
for n in lengths:
    length_counts[n - 1] += 1

In [18]:
length_counts

array([   1,    1,    2,    1,    2,    2,    5,   71,  406,  425,  688,
       1226, 2220, 3020, 3098, 2744, 1925, 1175,  694,  467,  299,  176,
        130,   92,   61,   16,   21,   16,   13,   10,   11,   10,    6,
          5,    4,    5,    5,    3,    2,    1,    1,    1,    5,    1,
          1,    2])

In [19]:
length_probs = length_counts / length_counts.sum()

In [20]:
length_probs

array([5.24383849e-05, 5.24383849e-05, 1.04876770e-04, 5.24383849e-05,
       1.04876770e-04, 1.04876770e-04, 2.62191924e-04, 3.72312533e-03,
       2.12899843e-02, 2.22863136e-02, 3.60776088e-02, 6.42894599e-02,
       1.16413214e-01, 1.58363922e-01, 1.62454116e-01, 1.43890928e-01,
       1.00943891e-01, 6.16151023e-02, 3.63922391e-02, 2.44887257e-02,
       1.56790771e-02, 9.22915574e-03, 6.81699004e-03, 4.82433141e-03,
       3.19874148e-03, 8.39014158e-04, 1.10120608e-03, 8.39014158e-04,
       6.81699004e-04, 5.24383849e-04, 5.76822234e-04, 5.24383849e-04,
       3.14630309e-04, 2.62191924e-04, 2.09753540e-04, 2.62191924e-04,
       2.62191924e-04, 1.57315155e-04, 1.04876770e-04, 5.24383849e-05,
       5.24383849e-05, 5.24383849e-05, 2.62191924e-04, 5.24383849e-05,
       5.24383849e-05, 1.04876770e-04])

In [21]:
from pyensembl import ensembl_grch38


In [30]:
genes = [gene for gene in ensembl_grch38.genes() if gene.biotype == "protein_coding"]

In [35]:
genes[0].transcripts

[Transcript(transcript_id=ENST00000612152, name=TSPAN6-204, gene_id=ENSG00000000003, gene_name=TSPAN6, biotype=protein_coding, location=X:100627109-100637104),
 Transcript(transcript_id=ENST00000373020, name=TSPAN6-201, gene_id=ENSG00000000003, gene_name=TSPAN6, biotype=protein_coding, location=X:100628670-100636806),
 Transcript(transcript_id=ENST00000614008, name=TSPAN6-205, gene_id=ENSG00000000003, gene_name=TSPAN6, biotype=protein_coding, location=X:100632063-100637104),
 Transcript(transcript_id=ENST00000496771, name=TSPAN6-203, gene_id=ENSG00000000003, gene_name=TSPAN6, biotype=processed_transcript, location=X:100632541-100636689),
 Transcript(transcript_id=ENST00000494424, name=TSPAN6-202, gene_id=ENSG00000000003, gene_name=TSPAN6, biotype=processed_transcript, location=X:100633442-100639991)]

In [36]:
genes[0].transcripts[0].protein_sequence

'MLKLYAMFLTLVFLVELVAAIVGFVFRHEIKNSFKNNYEKALKQYNSTGDYRSHAVDKIQNTLHCCGVTDYRDWTDTNYYSEKGFPKSCCKLEDCTPQRDADKVNNELIGIFLAYCLSRAITNNQYEIV'

In [44]:
def concat_protein_sequences(genome=ensembl_grch38, _cache={}):
    if genome in _cache:
        return _cache[genome]
    genes = [gene for gene in genome.genes() if gene.biotype == "protein_coding"]
    all_protein_sequences = []
    for gene in genes:
        curr_transcripts = [
            t for t in gene.transcripts if t.biotype == "protein_coding" 
        ]
        protein_sequences = [
            t.protein_sequence for t in curr_transcripts 
            if  t.protein_sequence is not None 
            and "*" not in t.protein_sequence
        ]
        if len(protein_sequences) == 0:
            continue 
        all_protein_sequences.append(max(protein_sequences, key=len))
    long_str =  "".join(all_protein_sequences)
    _cache[genome] = long_str
    return long_str

all_proteins = concat_protein_sequences()

In [45]:
len(all_proteins)

11399522

In [65]:
def generate_decoys_for_allele(
        hit_seqs, 
        max_length=None,
        decoys_per_hit=100, 
        genome=ensembl_grch38,
        length_pseudocount=2):
    n_hits = len(hit_seqs)
    lengths = [len(p) for p in hit_seqs]
    lengths_set = set(lengths)
    if max_length is None:
        max_length = max(lengths_set)
    length_counts = np.ones(max_length, dtype=int) * length_pseudocount
    for n in lengths:
        length_counts[n - 1] += 1
    length_probs = length_counts / length_counts.sum()
    
    n_decoys = n_hits * decoys_per_hit
    long_protein_str = concat_protein_sequences(genome)
    max_pos = len(long_protein_str)
    random_indices = np.random.randint(0, max_pos, n_decoys, dtype="int64")
    
    random_lengths = np.random.choice(
        a=1 + np.arange(max_length),
        size=n_decoys,
        replace=True,
        p=length_probs)
    decoys = []
    for i, n  in zip(random_indices, random_lengths):
        decoys.append(long_protein_str[i:i+n])
    return decoys

def generate_decoys(df_hits, decoys_per_hit=200, genome=ensembl_grch38, length_pseudocount=2):
    columns = {"allele": [], "peptide": []}
    max_length = df_hits.peptide.str.len().max()
    for allele, df_allele in df_hits.groupby("inferred_allele"):
        print("%s (%d)" % (allele, len(df_allele)))
        decoys = generate_decoys_for_allele(
            hit_seqs=set(df_allele.peptide),
            max_length=max_length,
            decoys_per_hit=decoys_per_hit,
            genome=genome,
            length_pseudocount=length_pseudocount)
        decoy_lengths = [len(p) for p in decoys]
        print("==> generated %d decoys (min length=%d, max length=%d, mean length=%f)" % (
            len(decoys),
            min(decoy_lengths),
            max(decoy_lengths),
            np.mean(decoy_lengths)))
        
        columns["allele"].extend([allele] * len(decoys))
        columns["peptide"].extend(decoys)
    df_decoys = pd.DataFrame(columns)
    df_decoys.to_csv("mhc2-decoys-7-31-2018.csv", index=False)

In [66]:
generate_decoys(df_hits=hits)

HLA-DPA1*01:03/DPB1*02:01 (12)
==> generated 2400 decoys (min length=1, max length=46, mean length=22.366250)
HLA-DPA1*01:03/DPB1*04:01 (4)
==> generated 800 decoys (min length=1, max length=46, mean length=22.865000)
HLA-DQA1*01:01/DQB1*05:01 (33)
==> generated 6600 decoys (min length=1, max length=46, mean length=21.809545)
HLA-DQA1*01:02/DQB1*03:01 (6)
==> generated 1200 decoys (min length=1, max length=46, mean length=22.800833)
HLA-DQA1*01:02/DQB1*05:01 (1)
==> generated 200 decoys (min length=1, max length=46, mean length=24.835000)
HLA-DQA1*01:02/DQB1*06:02 (55)
==> generated 10400 decoys (min length=1, max length=46, mean length=20.972404)
HLA-DQA1*01:02/DQB1*06:04 (1)
==> generated 200 decoys (min length=1, max length=46, mean length=22.825000)
HLA-DQA1*02:01/DQB1*02:02 (7355)
==> generated 1468200 decoys (min length=1, max length=46, mean length=14.777782)
HLA-DQA1*03:01/DQB1*02:01 (13)
==> generated 2600 decoys (min length=1, max length=46, mean length=23.320000)
HLA-DQA1*03

In [67]:
!head -n 1000 mhc2-decoys-7-31-2018.csv

allele,peptide
HLA-DPA1*01:03/DPB1*02:01,GTRKKIAYRKAV
HLA-DPA1*01:03/DPB1*02:01,DLLPVAQTEPSIWTVDDVWAFIHSLPGCQDIADE
HLA-DPA1*01:03/DPB1*02:01,VASRAPALGPAVRALGATFGPLLLRAPPPPSSPPPGGAPDGSEPSP
HLA-DPA1*01:03/DPB1*02:01,HMHHRHRHHHHHHHPPAGSALDPSYGPLLMP
HLA-DPA1*01:03/DPB1*02:01,SASRAGPAHAGHTAPMRPSYSAQEGLAGYQREGPHPAW
HLA-DPA1*01:03/DPB1*02:01,PPSLQTLPSPPATPPSQVPPTQLIMSFP
HLA-DPA1*01:03/DPB1*02:01,LQNSVK
HLA-DPA1*01:03/DPB1*02:01,EEYVHRIGRTGRVGNLGLATSFFNERNINITKDLL
HLA-DPA1*01:03/DPB1*02:01,YTSHLQLKEK
HLA-DPA1*01:03/DPB1*02:01,GLKS
HLA-DPA1*01:03/DPB1*02:01,LQTKGKNGDGRRRSAKDHHPGK
HLA-DPA1*01:03/DPB1*02:01,KLTNLNVDRNHLEALPPEIGGCVALSVLSLRDNRLAV
HLA-DPA1*01:03/DPB1*02:01,GKTKVEVAKMIQEVKGEVTIHYNKLQADPKQGMS
HLA-DPA1*01:03/DPB1*02:01,S
HLA-DPA1*01:03/DPB1*02:01,NRDIQEKREASLEESPV
HLA-DPA1*01:03/DPB1*02:01,AREKLAREEPDIYQIAKSEPRDAGTDQRSSGIIRLHTIKQI
HLA-DPA1*01:03/DPB1*02:01,EEEEVEEEEEEVVEEELVG
HLA-DPA1*01:03/DPB1*02:01,KTQRSPVRIPF
HLA-DPA1*01:03/DPB1*02:01,KFLTALAQDGVINEEALSVTELDRVYGGL
HLA-DPA1*01:03/DP

In [57]:
len("FLHRNIQEYLSILTDP")

16